In [8]:
! pip install sklearn
! pip install pandas
! pip install tqdm
! pip install -U sentence-transformers
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\adamm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement sentence_transformers==4.37.2 (from versions: 0.1.0, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.4.1, 0.2.5, 0.2.5.1, 0.2.6, 0.2.6.1, 0.2.6.2, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.5.1, 0.3.6, 0.3.7, 0.3.7.1, 0.3.7.2, 0.3.8, 0.3.9, 0.4.0, 0.4.1, 0.4.1.1, 0.4.1.2, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.1.0, 1.1.1, 1.2.0, 1.2.1, 2.0.0, 2.1.0, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.5.0, 2.5.1, 2.6.0, 2.6.1, 2.7.0)
ERROR: No matching distribution found for sentence_transformers==4.37.2

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\adamm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --up


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\adamm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import torch

from sentence_transformers import SentenceTransformer, losses, models, evaluation
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from torch import nn
from sklearn.metrics import matthews_corrcoef, f1_score, classification_report

from numpy.linalg import norm
import numpy as np
from tqdm import tqdm


Create drug-pairs dataset

In [2]:
df = pd.read_json("processed.json")
df.drop(columns=['DRUG_CID', "DRUG_NAME"], inplace=True)
df['INDEX'] = df.index
df1 = df.copy()
df2 = df.copy()
df1.columns = ['DRUG_ID1', 'INTERACTIONS1', 'STRUCTURE1', 'TARGET1', 'ENZYME1', 'PATH1', 'SIDE_EFFECTS1', 'INDICATIONS1', 'OFFSIDE_EFFECT1', 'INDEX1']
df2.columns = ['DRUG_ID2', 'INTERACTIONS2', 'STRUCTURE2', 'TARGET2', 'ENZYME2', 'PATH2', 'SIDE_EFFECTS2', 'INDICATIONS2', 'OFFSIDE_EFFECT2', 'INDEX2']
df = pd.merge(df1, df2, how="cross")
df = df[df['DRUG_ID1'] != df['DRUG_ID2']]
df.reset_index(inplace=True)
df['INTERACTION'] = df.apply(lambda row: row['INTERACTIONS1'][row['INDEX2']], axis=1)

df['id_pair'] = df[['DRUG_ID1', 'DRUG_ID2']].apply(lambda x: '-'.join(sorted(map(str, x))), axis=1)
df.drop_duplicates(subset='id_pair', inplace=True)
df.dropna(inplace=True)
df.drop(columns=["index", 'INTERACTIONS1', 'INTERACTIONS2', 'INDEX1', 'INDEX2', 'id_pair'], inplace=True)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

Create data for transformers saved individually for each feature in a dict

In [3]:
data = {}
to_edit = ['STRUCTURE', 'TARGET', 'ENZYME', 'PATH', 'SIDE_EFFECTS', 'INDICATIONS', 'OFFSIDE_EFFECT']
for column in to_edit:
    column1 = column + '1'
    column2 = column + '2'
    data[column] = [InputExample(texts=[" ".join(map(str, row[0])), " ".join(map(str, row[1]))], label=row[2])
                    for row in zip(df[column1], df[column2], df['INTERACTION'])]

Spit train and test data

In [7]:
split = int(len(data['STRUCTURE']) * 0.8)
structure_train_data = data['STRUCTURE'][:split]
structure_test_data = data['STRUCTURE'][split:]

In [12]:
word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

structure_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(structure_train_data, batch_size=4)
train_loss = losses.ContrastiveLoss(model=structure_model)

structure_model.fit([(train_dataloader, train_loss)],epochs=10)
structure_model.save("structure_model")


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19847 [00:00<?, ?it/s]

In [8]:
structure_model = SentenceTransformer("structure_model")
structure_texts = [example.texts for example in structure_test_data]
structure_labels = [example.label for example in structure_test_data]
structure_encodings = [structure_model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(structure_texts)]

100%|██████████| 19847/19847 [04:43<00:00, 70.12it/s]


In [11]:
structure_predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in structure_encodings]

structure_matthews = matthews_corrcoef(structure_labels, structure_predictions)
print("Matthews Correlation Coefficient:", structure_matthews)
print(classification_report(structure_labels, structure_predictions))


Matthews Correlation Coefficient: 0.76672989010064
              precision    recall  f1-score   support

           0       0.91      0.94      0.92     13216
           1       0.87      0.81      0.84      6631

    accuracy                           0.90     19847
   macro avg       0.89      0.88      0.88     19847
weighted avg       0.90      0.90      0.90     19847



In [13]:
split = int(len(data['TARGET']) * 0.8)
target_train_data = data['TARGET'][:split]
target_test_data = data['TARGET'][split:]

In [13]:
torch.cuda.empty_cache()

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

target_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(target_train_data, batch_size=16)
train_loss = losses.ContrastiveLoss(model=target_model)

target_model.fit([(train_dataloader, train_loss)],epochs=10)
target_model.save("target_model")


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

In [14]:
target_model = SentenceTransformer("target_model")
target_texts = [example.texts for example in target_test_data]
target_labels = [example.label for example in target_test_data]
target_encodings = [target_model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(target_texts)]

100%|██████████| 19847/19847 [03:43<00:00, 88.78it/s]


In [20]:
target_predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in target_encodings]

target_matthews = matthews_corrcoef(target_labels, target_predictions)
print("Matthews Correlation Coefficient:", target_matthews)
print(classification_report(target_labels, target_predictions))

Matthews Correlation Coefficient: 0.7027771026711205
              precision    recall  f1-score   support

           0       0.93      0.85      0.89     13216
           1       0.75      0.88      0.81      6631

    accuracy                           0.86     19847
   macro avg       0.84      0.86      0.85     19847
weighted avg       0.87      0.86      0.86     19847



In [22]:
split = int(len(data['ENZYME']) * 0.8)
enzyme_train_data = data['ENZYME'][:split]
enzyme_test_data = data['ENZYME'][split:]

In [14]:
torch.cuda.empty_cache()

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

enzyme_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(enzyme_train_data, batch_size=16)
train_loss = losses.ContrastiveLoss(model=enzyme_model)

enzyme_model.fit([(train_dataloader, train_loss)],epochs=10)
enzyme_model.save("enzyme_model")

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [91]:
enzyme_model = SentenceTransformer("enzyme_model")
enzyme_texts = [example.texts for example in enzyme_test_data]
enzyme_labels = [example.label for example in enzyme_test_data]
enzyme_encodings = [enzyme_model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(enzyme_texts)]

100%|██████████| 19847/19847 [03:50<00:00, 86.19it/s]


In [127]:
enzyme_predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in enzyme_encodings]

enzyme_matthews = matthews_corrcoef(enzyme_labels, enzyme_predictions)
print("Matthews Correlation Coefficient:", enzyme_matthews)
print(classification_report(enzyme_labels, enzyme_predictions))

Matthews Correlation Coefficient: 0.5469137315839356
              precision    recall  f1-score   support

           0       0.88      0.79      0.83     13216
           1       0.65      0.78      0.71      6631

    accuracy                           0.78     19847
   macro avg       0.76      0.78      0.77     19847
weighted avg       0.80      0.78      0.79     19847



In [33]:
split = int(len(data['PATH']) * 0.8)
path_train_data = data['PATH'][:split]
path_test_data = data['PATH'][split:]

In [9]:
torch.cuda.empty_cache()

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

path_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(path_train_data, batch_size=16)
train_loss = losses.ContrastiveLoss(model=path_model)

path_model.fit([(train_dataloader, train_loss)],epochs=10)
path_model.save("path_model")

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9924 [00:00<?, ?it/s]

In [34]:
path_model = SentenceTransformer("path_model")
path_texts = [example.texts for example in path_test_data]
path_labels = [example.label for example in path_test_data]
path_encodings = [path_model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(path_texts)]

100%|██████████| 19847/19847 [03:41<00:00, 89.60it/s]


In [39]:
path_predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in path_encodings]

path_matthews = matthews_corrcoef(path_labels, path_predictions)
print("Matthews Correlation Coefficient:", path_matthews)
print(classification_report(path_labels, path_predictions))

Matthews Correlation Coefficient: 0.6354708024774931
              precision    recall  f1-score   support

           0       0.87      0.89      0.88     13216
           1       0.78      0.73      0.75      6631

    accuracy                           0.84     19847
   macro avg       0.82      0.81      0.82     19847
weighted avg       0.84      0.84      0.84     19847



In [40]:
split = int(len(data['SIDE_EFFECTS']) * 0.8)
se_train_data = data['SIDE_EFFECTS'][:split]
se_test_data = data['SIDE_EFFECTS'][split:]

In [10]:
torch.cuda.empty_cache()

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

se_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(se_train_data, batch_size=16)
train_loss = losses.ContrastiveLoss(model=se_model)

se_model.fit([(train_dataloader, train_loss)],epochs=10)
se_model.save("se_model")

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

In [41]:
se_model = SentenceTransformer("se_model")
se_texts = [example.texts for example in se_test_data]
se_labels = [example.label for example in se_test_data]
se_encodings = [se_model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(se_texts)]

100%|██████████| 19847/19847 [04:38<00:00, 71.20it/s]


In [45]:
se_predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in se_encodings]

se_matthews = matthews_corrcoef(se_labels, se_predictions)
print("Matthews Correlation Coefficient:", se_matthews)
print(classification_report(se_labels, se_predictions))

Matthews Correlation Coefficient: 0.7843927384808446
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     13216
           1       0.82      0.89      0.86      6631

    accuracy                           0.90     19847
   macro avg       0.88      0.90      0.89     19847
weighted avg       0.90      0.90      0.90     19847



In [4]:
split = int(len(data['INDICATIONS']) * 0.8)
indications_train_data = data['INDICATIONS'][:split]
indications_test_data = data['INDICATIONS'][split:]

In [5]:
torch.cuda.empty_cache()

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

indications_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(indications_train_data, batch_size=16)
train_loss = losses.ContrastiveLoss(model=indications_model)

indications_model.fit([(train_dataloader, train_loss)],epochs=10)
indications_model.save("indications_model")

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4962 [00:00<?, ?it/s]

In [48]:
indications_model = SentenceTransformer("indications_model")
indications_texts = [example.texts for example in indications_test_data]
indications_labels = [example.label for example in indications_test_data]
indications_encodings = [indications_model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(indications_texts)]

100%|██████████| 19847/19847 [03:39<00:00, 90.28it/s] 


In [52]:
indications_predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in indications_encodings]

indications_matthews = matthews_corrcoef(indications_labels, indications_predictions)
print("Matthews Correlation Coefficient:", indications_matthews)
print(classification_report(indications_labels, indications_predictions))

Matthews Correlation Coefficient: 0.7736557351589848
              precision    recall  f1-score   support

           0       0.95      0.89      0.92     13216
           1       0.81      0.90      0.85      6631

    accuracy                           0.90     19847
   macro avg       0.88      0.90      0.89     19847
weighted avg       0.90      0.90      0.90     19847



In [65]:
encodings = []
for model_path, test_data in [("structure_model", structure_test_data), ("target_model", target_test_data), 
                    ("enzyme_model", enzyme_test_data), ("path_model", path_test_data),
                    ("se_model", se_test_data), ("indications_model", indications_test_data)]:
    model = SentenceTransformer(model_path)
    texts = [example.texts for example in test_data]
    encodings.append([model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(texts)])

100%|██████████| 19847/19847 [03:43<00:00, 88.85it/s]


In [93]:
concatenated_encodings = [[np.array([]), np.array([])] for _ in encodings[0]]
for model_result in encodings:
    for i in range(len(model_result)):
        concatenated_encodings[i][0] = np.append(concatenated_encodings[i][0], model_result[i][0])
        concatenated_encodings[i][1] = np.append(concatenated_encodings[i][1], model_result[i][1])

In [128]:
predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.65 else 0 for e in concatenated_encodings]
labels = [example.label for example in structure_test_data]

matthews = matthews_corrcoef(labels, predictions)
print("Matthews Correlation Coefficient:", matthews)
print(classification_report(labels, predictions))

Matthews Correlation Coefficient: 0.7873705850191058
              precision    recall  f1-score   support

           0       0.95      0.90      0.92     13216
           1       0.82      0.90      0.86      6631

    accuracy                           0.90     19847
   macro avg       0.89      0.90      0.89     19847
weighted avg       0.91      0.90      0.90     19847



In [122]:
cos_sims = np.array([(i, np.dot(concatenated_encodings[i][0], concatenated_encodings[i][1]) / (norm(concatenated_encodings[i][0]) * norm(concatenated_encodings[i][1]))) for i in range(len(concatenated_encodings))])
new_preds = cos_sims[[label==0 for label in labels]]
sorted_preds = new_preds[new_preds[:, 1].argsort()][::-1][:10]
indices = list(sorted_preds[:, 0])
indices = [int(index + len(structure_train_data))for index in indices]
df.iloc[indices]

,DRUG_ID1,STRUCTURE1,TARGET1,ENZYME1,PATH1,SIDE_EFFECTS1,INDICATIONS1,OFFSIDE_EFFECT1,DRUG_ID2,STRUCTURE2,TARGET2,ENZYME2,PATH2,SIDE_EFFECTS2,INDICATIONS2,OFFSIDE_EFFECT2,INTERACTION
79763,DB00322,"[9, 10, 11, 14, 15, 18, 19, 20, 23, 143, 146, ...",[P04818],"[P19971, P11712]","[hsa00240, hsa00670, hsa01100, hsa01523]","[C0000737, C0000833, C0001824, C0002170, C0002...",[C0001418],"[10065553, 10023126, 10043554, 10016288, 10029...",DB01101,"[0, 9, 10, 11, 14, 15, 18, 19, 20, 23, 143, 14...",[P04818],"[P19971, P23141, Q12882, P32320, P11712]","[hsa00240, hsa00670, hsa01100, hsa01523]","[C0000731, C0000737, C0000768, C0001824, C0002...","[C0006142, C0006826, C0009402, C0020433, C0027...","[10022972, 10034695, 10002034, 10002064, 10052...",0
85521,DB00544,"[9, 10, 14, 15, 18, 19, 23, 178, 183, 184, 283...",[P04818],"[P05177, P19971, P11712, Q12882, Q16831, O9504...","[hsa00240, hsa00670, hsa01100, hsa01523]","[C0000768, C0000786, C0001824, C0002170, C0002...","[C0006079, C0006826, C0007097, C0007114, C0007...","[10022972, 10034695, 10002034, 10002064, 10002...",DB01101,"[0, 9, 10, 11, 14, 15, 18, 19, 20, 23, 143, 14...",[P04818],"[P19971, P23141, Q12882, P32320, P11712]","[hsa00240, hsa00670, hsa01100, hsa01523]","[C0000731, C0000737, C0000768, C0001824, C0002...","[C0006142, C0006826, C0009402, C0020433, C0027...","[10022972, 10034695, 10002034, 10002064, 10052...",0
87088,DB00322,"[9, 10, 11, 14, 15, 18, 19, 20, 23, 143, 146, ...",[P04818],"[P19971, P11712]","[hsa00240, hsa00670, hsa01100, hsa01523]","[C0000737, C0000833, C0001824, C0002170, C0002...",[C0001418],"[10065553, 10023126, 10043554, 10016288, 10029...",DB00544,"[9, 10, 14, 15, 18, 19, 23, 178, 183, 184, 283...",[P04818],"[P05177, P19971, P11712, Q12882, Q16831, O9504...","[hsa00240, hsa00670, hsa01100, hsa01523]","[C0000768, C0000786, C0001824, C0002170, C0002...","[C0006079, C0006826, C0007097, C0007114, C0007...","[10022972, 10034695, 10002034, 10002064, 10002...",0
87800,DB00289,"[9, 10, 11, 12, 14, 18, 178, 179, 185, 186, 25...","[P23975, P31645, Q01959]","[P10635, P33261, P08684]","[hsa04726, hsa04728, hsa05012, hsa05030, hsa05...","[C0000737, C0001807, C0003123, C0003467, C0003...","[C0004936, C0009421, C0011206, C0014544, C0015...","[10034695, 10002034, 10002065, 10052293, 10002...",DB01174,"[9, 10, 11, 14, 15, 18, 19, 178, 179, 180, 181...","[P14867, P43681, P36544, P42262, Q13002]","[P33261, P11712, P20813, P10632, P08684, P0517...","[hsa04080, hsa04723, hsa04727, hsa04742, hsa05...","[C0000768, C0002888, C0002994, C0003578, C0004...","[C0003467, C0008354, C0008489, C0009952, C0013...","[10022972, 10034695, 10002034, 10002064, 10002...",0
90025,DB00530,"[9, 10, 11, 12, 14, 15, 18, 19, 20, 178, 179, ...","[P00533, O75469]","[P08684, P20815, P05177, P04798, P10635, P1063...","[hsa01521, hsa01522, hsa04010, hsa04012, hsa04...","[C0000737, C0002170, C0002878, C0003123, C0003...","[C0006826, C0007131, C0027651, C0280100, C0346...","[10022972, 10034695, 10002034, 10002064, 10048...",DB06626,"[9, 10, 11, 12, 14, 15, 16, 18, 33, 143, 145, ...","[P17948, P35968, P35916]","[P08684, P20815, P05177, P33261, P22309]","[hsa04014, hsa04015, hsa04060, hsa04066, hsa04...","[C0000737, C0002170, C0002871, C0003862, C0004...",[C0007134],"[10002034, 10058116, 10065553, 10033661, 10021...",0
80502,DB00530,"[9, 10, 11, 12, 14, 15, 18, 19, 20, 178, 179, ...","[P00533, O75469]","[P08684, P20815, P05177, P04798, P10635, P1063...","[hsa01521, hsa01522, hsa04010, hsa04012, hsa04...","[C0000737, C0002170, C0002878, C0003123, C0003...","[C0006826, C0007131, C0027651, C0280100, C0346...","[10022972, 10034695, 10002034, 10002064, 10048...",DB08828,"[9, 10, 11, 12, 14, 15, 18, 19, 33, 37, 38, 17...",[Q99835],"[P11712, P08684, P10632, P33261]","[hsa04340, hsa04360, hsa05200, hsa05205, hsa05...","[C0000737, C0002170, C0002453, C0003862, C0004...",[C0007117],"[10002034, 10065553, 10020578, 10023126, 10028...",0
91899,DB01196,"[0, 9, 10, 11, 12, 14, 18, 19, 37, 38, 143, 14...","[Q92731, P03372, 